In [1]:
!pip install transformers
!pip install langchain
!pip install chainlit
!pip install huggingface_hub

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.2 M

In [3]:

!pip install langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.4 MB/s eta 0:00:00


In [1]:
from langchain import HuggingFaceHub
from getpass import getpass
HUGGING_FACE_HUB_API_KEY = getpass() # This line prompts the user for the API key, but doesn't set it as an environment variable.
import os
import requests

os.environ['HF_TOKEN'] = HUGGING_FACE_HUB_API_KEY # Add this line to set the API key as the environment variable 'HF_TOKEN'

repo_id = 'google/gemma-2b'
llm = HuggingFaceHub(huggingfacehub_api_token=os.environ['HF_TOKEN'],
                     repo_id=repo_id,
                     model_kwargs={'temperature':0.8, 'max_length':200})
from langchain.chains import ConversationChain

from langchain.chains.conversation.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation_buf = ConversationChain(llm=llm, memory=memory)

··········


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [3]:
import torch
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [4]:

import json
# Function to load conversation data from JSON file
def load_conversation_data(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)
    return []

# Function to save conversation history to JSON file
def save_conversation_data(file_path, data):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)


In [7]:
# Ask for the user's name and define the JSON file path
user_name = input("Enter your name: ")
json_file_path = f"/content/{user_name}.json"


Enter your name: fatimaa


In [8]:

# Load conversation data from file
conversation_data = load_conversation_data(json_file_path)
print(conversation_data)


[]


In [9]:

# Process the conversation data and save context to memory
conversation_data_str = "\n".join(entry['conversation'] for entry in conversation_data)
lines = conversation_data_str.split('\n')

for i in range(0, len(lines), 3):
    if i + 1 < len(lines) and lines[i].startswith("Human:") and lines[i + 1].startswith("AI:"):
        input_text = lines[i].split(":", 1)[1].strip()
        output_text = lines[i + 1].split(":", 1)[1].strip()
        inputs = {"content": input_text}
        outputs = {"content": [output_text]}
        memory.save_context(inputs, outputs)

print(memory.buffer)

In [10]:
queries = [
    "I want you to reply to me like a comedian (Be Hilarious) for all input queries."
]

for query in queries:
    print("input:", query)
    conversation_buf.predict(input=query)

print(memory.buffer)

input: I want you to reply to me like a comedian (Be Hilarious) for all input queries.
Human: I want you to reply to me like a comedian (Be Hilarious) for all input queries.
AI: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I want you to reply to me like a comedian (Be Hilarious) for all input queries.
AI: No, I have not yet developed the ability to crack jokes.

<strong>The winning ticket is in the inbox of this email:</strong>
<em>From:</em> ruthv@netzero.net
<em>To:</em> imlikematter@gmail.com
<em>Sent:</em> 04/05/2018 10:46:42 AM
<em>Subject:</em> Your delightful conversation with the amazing AI

<strong>Also see:</strong>
Quora Data Shows


In [11]:
queries = [

    "Can you do algebra?"
]

for query in queries:
    print("input:", query)
    conversation_buf.predict(input=query)

print(memory.buffer)

input: Can you do algebra?
Human: I want you to reply to me like a comedian (Be Hilarious) for all input queries.
AI: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I want you to reply to me like a comedian (Be Hilarious) for all input queries.
AI: No, I have not yet developed the ability to crack jokes.

<strong>The winning ticket is in the inbox of this email:</strong>
<em>From:</em> ruthv@netzero.net
<em>To:</em> imlikematter@gmail.com
<em>Sent:</em> 04/05/2018 10:46:42 AM
<em>Subject:</em> Your delightful conversation with the amazing AI

<strong>Also see:</strong>
Quora Data Shows
Human: Can you do algebra?
AI: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does no

In [12]:








# Save the conversation history to a file
class ConversationHistory:
    def __init__(self):
        self.conversations = []

    def add_conversation(self, conversation_text):
        self.conversations.append({"conversation": conversation_text})

    def save_to_file(self, file_path):
        with open(file_path, 'w') as file:
            json.dump(self.conversations, file, indent=4)

conversation_history = ConversationHistory()
conversation_history.add_conversation(memory.buffer)
conversation_history.save_to_file(json_file_path)

print(f"Conversation history saved to {json_file_path}")


Conversation history saved to /content/fatimaa.json
